In [1]:
import pandas as pd
import numpy as np
from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad

import tqdm.notebook as tq

import fiona
import geopandas as gpd
from geopandas import GeoDataFrame

from shapely.geometry import Polygon, MultiPolygon

import os
import multiprocessing as mp

import warnings
warnings.filterwarnings("ignore")


In [13]:
# set path to shape file
# shape_file_path = '/home/dima/Coding/education/HSI/aspirantura/Dissertation/russia_data/openf_gauges_watersheds/watersheds_openf.shp'
shape_file_path = '/home/dima/Coding/education/HSI/aspirantura/Dissertation/conus_data/basin_set_full_res/HCDN_nhru_final_671.shp'
# set path to downloaded HydroATLAS
gdb_file_path = '/home/dima/Coding/education/HSI/aspirantura/Dissertation/files/BasinATLAS/BasinATLAS_v10.gdb/'
# set path where results will be stored
path_to_save = '/home/dima/Coding/education/HSI/aspirantura/Dissertation/conus_data/featureXtractor'

conus = True


In [3]:
# Read shape file with geometry column
big_shape = gpd.read_file(shape_file_path)
try:
    big_shape = big_shape[['code', 'geometry', 'area_1']]
    # rename column of gauge identification number to capital ID
    big_shape = big_shape.rename(columns={"code": "gauge_id"})
except KeyError:
    big_shape = big_shape[['hru_id', 'geometry', 'AREA']]
    # rename column of gauge identification number to capital ID
    big_shape = big_shape.rename(columns={"hru_id": "gauge_id",
                                          'AREA': 'area'})

if conus:
    big_shape['gauge_id'] = ['0'+i if len(i) != 8
                             else i
                             for i in map(str, big_shape['gauge_id'])]
else:
    big_shape['gauge_id'] = big_shape['gauge_id'].apply(lambda x: str(int(x)))

big_shape


,gauge_id,geometry,area
0,1013500,"MULTIPOLYGON (((-68.35650 46.90311, -68.35612 ...",2.303988e+09
1,1022500,"POLYGON ((-67.97836 44.61310, -67.97800 44.613...",6.203873e+08
2,1030500,"MULTIPOLYGON (((-67.83991 45.36614, -67.83955 ...",3.676155e+09
3,1031500,"MULTIPOLYGON (((-69.33810 45.12317, -69.33800 ...",7.665447e+08
4,1047000,"POLYGON ((-70.10847 45.21669, -70.10858 45.216...",9.049562e+08
...,...,...,...
666,14309500,"POLYGON ((-123.81322 42.89103, -123.81312 42.8...",2.263143e+08
667,14316700,"POLYGON ((-122.49936 43.47688, -122.49972 43.4...",5.880250e+08
668,14325000,"POLYGON ((-124.07751 42.89822, -124.07716 42.8...",4.449257e+08
669,14362250,"POLYGON ((-123.15128 42.19624, -123.15118 42.1...",4.387790e+07


In [4]:
def polygon_area(lats: np.ndarray, lons: np.ndarray, radius=6378137):
    """
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified.
    Otherwise, in the units of provided radius.
    lats and lons are in degrees.

    Args:
        lats (list): list of latitudinal coordinates
        lons (list): list of longitudinal coordinates
        radius (int, optional): Earth radius. Defaults to 6378137.

    Returns:
        area: area of object in sq. km
    """
    
    lats, lons = np.deg2rad(lats), np.deg2rad(lons)

    # Line integral based on Green's Theorem, assumes spherical Earth

    #close polygon
    if lats[0] != lats[-1]:
        lats = append(lats, lats[0])
        lons = append(lons, lons[0])

    #colatitudes relative to (0,0)
    a = sin(lats/2)**2 + cos(lats) * sin(lons/2)**2
    colat = 2*arctan2(sqrt(a), sqrt(1-a))

    #azimuths relative to (0,0)
    az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

    # Calculate diffs
    # daz = diff(az) % (2*pi)
    daz = diff(az)
    daz = (daz + pi) % (2 * pi) - pi

    deltas = diff(colat)/2
    colat = colat[0:-1]+deltas

    # Perform integral
    integrands = (1-cos(colat)) * daz

    # Integrate
    area = abs(sum(integrands))/(4*pi)

    area = min(area, 1-area)
    if radius is not None:  # return in units of radius
        return area * 4 * pi * radius**2 / 10**6
    else:  # return in ratio of sphere total area
        return area / 10**6


def select_big_from_MP(WS_geometry):
    """
    Select biggest polygon from MultiPolygon object.
    Needs to eliminate fake structures out of basin shape

    Args:
        WS_geometry (Geometry): Desired basin shape

    Returns:
        Ws_geometry: Biggest polygon which correspond to natural form
    """
    if type(WS_geometry) == MultiPolygon:
        big_area = [polygon_area(lats=polygon.exterior.coords.xy[1],
                                 lons=polygon.exterior.coords.xy[0])
                    for polygon in WS_geometry]
        WS_geometry = WS_geometry[np.argmax(big_area)]
    else:
        WS_geometry = WS_geometry
    return WS_geometry


def get_gdf_Poly(some_geometry: Polygon):
    """
    Transform Polygon object to GeoDataFrame with EPSG projection

    Args:
        some_geometry (Polygon): Polygon from GeoDataFrame of Polygon's

    Returns:
        target (GeoDataFrame): Polygon stored in GeoDataFrame format
    """
    target = gpd.GeoSeries(
        select_big_from_MP(some_geometry))
    target = gpd.GeoDataFrame({'geometry': target}).set_crs('EPSG:4326')

    return target


def find_POLY_area(poly):
    """
    Calculates area of shape stored in GeoDataFrame

    Args:
        poly (GeoDataFrame): Desired shape

    Returns:
        area (float): area of object in sq. km
    """
    if poly.empty:
        return np.NaN
    else:
        poly = select_big_from_MP(poly['geometry'][0])
        area = polygon_area(lats=poly.exterior.xy[1],
                            lons=poly.exterior.xy[0])
    return area


def parallelize_function(WS: GeoDataFrame, path_to_HydroATLAS: str):
    """
    This function generate list of tuples
    where each tuple stands for row in DF
    of watersheds

    Args:
        WS (GeoDataFrame): GeoDataFrame of desired watersheds
        path_to_HydroATLAS (str): path to gdb of HydroATLAS on disk
        layer_small (fiona): [description]

    Returns:
        mp_tuples (tuple): tuple of values which will be required for 
        parallel launch
    """
    mp_tuples = list()

    for row in range(len(WS)):
        mp_tuples.append((WS.loc[row, 'geometry'],
                          path_to_HydroATLAS))

    return mp_tuples


In [5]:
# Necessary columns to work with HydroATLAS
monthes = ['01', '02', '03', '04', '05', '06',
           '07', '08', '09', '10', '11', '12']
lc_classes = ['01', '02', '03', '04', '05', '06', '07', '08', '09',
              '10', '11', '12', '13', '14', '15', '16', '17', '18',
                    '19', '20', '21', '22']
wetland_classes = ['01', '02', '03', '04', '05', '06', '07', '08', '09']
hydrology_variables = [item for sublist in [['inu_pc_ult'],
                                            ['lka_pc_use'],
                                            ['lkv_mc_usu'],
                                            ['rev_mc_usu'],
                                            ['dor_pc_pva'],
                                            ['gwt_cm_sav']]
                       for item in sublist]
physiography_variables = [item for sublist in [['ele_mt_sav'],
                                               ['slp_dg_sav'],
                                               ['sgr_dk_sav']]
                          for item in sublist]
climate_variables = [item for sublist
                     in [['clz_cl_smj'],
                         ['cls_cl_smj'],
                         ['tmp_dc_s{}'.format(i) for i in monthes],
                         ['pre_mm_s{}'.format(i) for i in monthes],
                         ['pet_mm_s{}'.format(i) for i in monthes],
                         ['aet_mm_s{}'.format(i) for i in monthes],
                         ['ari_ix_sav'],
                         ['cmi_ix_s{}'.format(i) for i in monthes],
                         ['snw_pc_s{}'.format(i) for i in monthes]]
                     for item in sublist]
landcover_variables = [item for sublist
                       in [['glc_cl_smj'],
                           ['glc_pc_s{}'.format(i) for i in lc_classes],
                           ['wet_pc_s{}'.format(i)
                            for i in wetland_classes],
                           ['for_pc_sse'],
                           ['crp_pc_sse'],
                           ['pst_pc_sse'],
                           ['ire_pc_sse'],
                           ['gla_pc_sse'],
                           ['prm_pc_sse']]
                       for item in sublist]
soil_and_geo_variables = [item for sublist
                          in [['cly_pc_sav'],
                              ['slt_pc_sav'],
                              ['snd_pc_sav'],
                              ['soc_th_sav'],
                              ['swc_pc_syr'],
                              ['swc_pc_s{}'.format(i) for i in monthes],
                              ['kar_pc_sse'],
                              ['ero_kh_sav']]
                          for item in sublist]
urban_variables = [item for sublist in [['urb_pc_sse'],
                                        ['hft_ix_s93'],
                                        ['hft_ix_s09']]
                   for item in sublist]
majority_class = [item for sublist in [['glc_cl_smj'],
                                       ['clz_cl_smj'],
                                       ['cls_cl_smj']]
                   for item in sublist]


In [6]:
def featureXtractor(user_ws: Polygon, gdb_file_path: str):
    """
    Fucntion calculates weighted mean of variables which are occured
    in intersection of user_ws and subbasins from HydroATLAS database

    Args:
        user_ws (Polygon): User's watershed boundary
        gdb_file_path (str): path to gdb of HydroATLAS on disk

    Returns:
        geo_vector (Series): Series of variables corresponded to user_ws id
    """

    # get only biggest polygon areas from watershed
    gdf_your_WS = select_big_from_MP(user_ws)

    # transform to geopandas for geometry operations
    gdf_your_WS = gpd.GeoSeries([gdf_your_WS])
    gdf_your_WS = gpd.GeoDataFrame({'geometry': gdf_your_WS})
    gdf_your_WS = gdf_your_WS.set_crs('EPSG:4326')

    # connect to HydroATLAS file with fiona+gpd interface
    layer_small = fiona.listlayers(gdb_file_path)[-1]
    # Read choosen geodatabase layer with geopandas
    gdf = gpd.read_file(gdb_file_path,
                        mask=user_ws,
                        layer=layer_small,
                        ignore_geometry=False)
    # create new column where each intersection is geodataframe
    gdf['gdf_geometry'] = tuple(map(get_gdf_Poly, gdf['geometry']))
    # calculate weight of each intersection correspond to it native size
    gdf['weights'] = gdf['gdf_geometry'].apply(
        lambda x: gpd.overlay(gdf_your_WS, x)).apply(find_POLY_area) /\
        gdf['gdf_geometry'].apply(find_POLY_area)
    # calculate area with weight appliance
    gdf['weight_area'] = tuple(map(find_POLY_area,
                                   gdf['gdf_geometry'])) * gdf['weights']

    geo_vector = pd.DataFrame(index=[0])

    for major_class in majority_class:
        geo_vector[major_class] = np.bincount(gdf[major_class],
                                              weights=gdf['weights']).argmax()

    for agg_class in np.setdiff1d(hydrology_variables +
                                  physiography_variables +
                                  climate_variables +
                                  landcover_variables +
                                  soil_and_geo_variables +
                                  urban_variables,
                                  majority_class):

        geo_vector[agg_class] = np.sum(
            gdf[agg_class] * gdf['weights'])/np.sum(gdf['weights'])
    # calculate each variable weighted mean

    # some values in HydroATLAS was multiplied by <X>, so to bring it
    # back to original form this procedure is required
    divide_by_10 = [item for sublist
                    in [['lka_pc_use'],
                        ['dor_pc_pva'],
                        ['slp_dg_sav'],
                        ['tmp_dc_s{}'.format(i) for i in monthes],
                        ['hft_ix_s93'],
                        ['hft_ix_s09']]
                    for item in sublist]
    divide_by_100 = [item for sublist
                     in [['ari_ix_sav'],
                         ['cmi_ix_s{}'.format(i) for i in monthes]]
                     for item in sublist]

    geo_vector[divide_by_10] /= 10
    geo_vector[divide_by_100] /= 100
    # store basin area
    geo_vector['ws_area'] = find_POLY_area(gdf_your_WS)

    return geo_vector


In [7]:
output = list()
for ws in tq.tqdm(big_shape['geometry']):
    output.append(featureXtractor(user_ws=ws,
                                  gdb_file_path=gdb_file_path))


  0%|          | 0/671 [00:00<?, ?it/s]

In [10]:
def save_results(eXtracted_data: list, gauge_ids: pd.Series, path_to_save: str):

    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)

    # create DataFrame to save it by categories
    df_to_disk = pd.concat(eXtracted_data).set_index(gauge_ids)

    df_to_disk.to_pickle('{path}/{fn}.pkl'.format(path=path_to_save,
                                                  fn='geo_vector'))

    save_names = {'hydro': hydrology_variables,
                  'physio': physiography_variables+['ws_area'],
                  'climate': climate_variables,
                  'landcover': landcover_variables,
                  'soil_geo': soil_and_geo_variables,
                  'urban': urban_variables}

    for key, values in save_names.items():
        df_to_disk[values].to_pickle('{path}/{fn}.pkl'.format(path=path_to_save,
                                                              fn=key))

    return


In [16]:
path_to_save = '/home/dima/Coding/python_scripts/Russia_LSTM/Tranfer_learning_LSTM/data'
from pathlib import Path
Path(path_to_save).mkdir(exist_ok=True)

In [14]:
big_shape['gauge_id'] = ['0' + str(gauge) if len(str(gauge)) != 8
                           else str(gauge)
                           for gauge in big_shape['gauge_id']]
big_shape['gauge_id']

0      01013500
1      01022500
2      01030500
3      01031500
4      01047000
         ...   
666    14309500
667    14316700
668    14325000
669    14362250
670    14400000
Name: gauge_id, Length: 671, dtype: object

In [15]:
save_results(eXtracted_data=output,
             gauge_ids=big_shape['gauge_id'],
             path_to_save=path_to_save)
